In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!rm -r /root/.ngrok2

In [20]:
!mkdir /root/.ngrok2 && echo "authtoken: 2QBHwdCEScD1khrjiKTpZm7pXPc_6CVfSn5wdNudo15fXo9EN" > /root/.ngrok2/ngrok.yml

In [30]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-gener2",
)


import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()



from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)



def inference_fn(
        prompt,
        min_length=10,
        max_length=30,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }



from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-gener2/epoch=2-val_loss=0.95.ckpt


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


 * Serving Flask app 'ratsnlp.nlpbook.generation.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ef67-34-70-158-42.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:32:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:32:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:03] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:07] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:11] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:14] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:18] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:21] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:23] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:33:26] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Jun/2023 14:35:03] "POST /api HTTP/1.1" 200 -
